In [6]:
%env kubectl=/usr/bin/kubectl
%env istio=/Users/gsjurseth/istio-1.0.4
%env istioctl=/Users/gsjurseth/istio-1.0.4/bin/istioctl
%env ingress=35.225.233.232
%env demo=/Users/gsjurseth/dev/fazio/microservices-demo

env: kubectl=/usr/bin/kubectl
env: istio=/Users/gsjurseth/istio-1.0.4
env: istioctl=/Users/gsjurseth/istio-1.0.4/bin/istioctl
env: ingress=35.225.233.232
env: demo=/Users/gsjurseth/dev/fazio/microservices-demo


---
## Fazio Retail Shop UI

http://35.225.233.232/


Let's look at the running pods

In [8]:
!$kubectl get pods

NAME                                     READY   STATUS    RESTARTS   AGE
adservice-6578bcfd8f-lgfgf               2/2     Running   0          2h
cartservice-5cb655594d-bjf68             2/2     Running   4          2h
checkoutservice-799869764-lzpt2          2/2     Running   0          2h
currencyservice-6cbdc7768d-c7xvg         2/2     Running   0          2h
emailservice-9b7867fcb-n99kw             2/2     Running   0          2h
frontend-blue-6fb5d8d967-bvrbs           2/2     Running   0          2h
frontend-gray-5447f56f9b-n8kc2           2/2     Running   0          2h
loadgenerator-cf64757b5-lslx2            2/2     Running   0          2h
paymentservice-57cd6d499f-dwzj6          2/2     Running   0          2h
productcatalogservice-584d6ff79d-jrgj5   2/2     Running   0          2h
recommendationservice-69869d97b9-h5rbb   2/2     Running   0          2h
redis-cart-9b864d47f-2nzdz               2/2     Running   0          2h
shippingservice-7b7759d995-ckqjc         2/2     R

---
## Architectural Diagram

![architecture](https://raw.githubusercontent.com/theganyo/microservices-demo/fazio-playground/img/architecture-diagram.jpeg)

---
## I've heard that blue is hip color!

But is it really? Let's canary test a blue front-end. We'll route 20% of traffic to the blue version.

In [9]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: frontend-ingress
spec:
  gateways:
  - frontend-gateway
  hosts:
  - '*'
  http:
  - route:
    - destination:
        host: frontend
        subset: gray
      weight: 66
    - destination:
        host: frontend
        subset: blue
      weight: 34

EOF

virtualservice.networking.istio.io/frontend-ingress configured


---
## It turns out, customers prefer grey.

Let's revert that change.

In [10]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: frontend-ingress
spec:
  gateways:
  - frontend-gateway
  hosts:
  - '*'
  http:
  - route:
    - destination:
        host: frontend
        subset: gray
      weight: 100

EOF

virtualservice.networking.istio.io/frontend-ingress configured


---
## Jaeger UI

Run in console:

    kubectl port-forward -n istio-system $(kubectl get pod -n istio-system -l app=jaeger -o jsonpath='{.items[0].metadata.name}') 16686:16686 &
    
http://penguin.linux.test:16686/search?service=emailservice

---
## Grafana UI

Run in console:

    kubectl -n istio-system port-forward $(kubectl -n istio-system get pod -l app=grafana -o jsonpath='{.items[0].metadata.name}') 3000:3000 &
    
http://penguin.linux.test:3000/d/3/pilot-dashboard